# Automated UAS SkyWatch Standard Format Remark Functions & Script

## Origin Location: RWY
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** October 29, 2021

***
***How to Use:***

`The main() function will output the dataframe containing the complete UAS location information for Standard Format Remarks referencing a Runway (RWY). To call the main() function, run the entire notebook and enter the file path for the .csv containing potential runway records enclosed in single quotation marks: main('file_path')`
***

## Import Statements

In [3]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

# REMARK MUST REFERENCE A RUNWAY

### uas_rwy() 

This function determines the records for which the Remarks reference a Runway.

In [1]:
def uas_rwy(file):
    '''
    This function takes in a file (str) containing unidentified Remarks from SkyWatch and determines those that reference a 
    runway as the origin location for the UAS sighting. This function outputs a dataframe containing the Remark, the Runway 
    Location, the Reporting Facility (needed to find the airport in relation to the runway referenced), and the UAS Distance
    from the runway (if provided)
    
    @param file: a .csv file containing unidentified Remarks from SkyWaych
    '''
    
    # Read in .csv file provided by user
    unid_remarks = pd.read_csv(file)
    
    # Create List that contains each Standard Remark, and the Heading/Direction
    # information contained in each remark

    remark_uas_loc = []
    remarks = unid_remarks['REMARKS']

    # regular expression for any heading/direction
    rwy_regex = 'RWY\s?[0-9]*[A-Z]?'

    # Loop through all remarks and search for the heading/direction regex above
    for i in range(len(remarks)):
        rwy = re.findall(rwy_regex, remarks[i])
        remark_uas_loc.append(remarks[i])
        remark_uas_loc.append(rwy)

    # Split Remarks and Heading/Directions into two seperate lists and create
    # pandas dataframe
    remark = []
    uas_loc = []

    for i in range(0, len(remark_uas_loc), 2):
        remark.append(remark_uas_loc[i])
        uas_loc.append(remark_uas_loc[i+1])

    remark_uas_loc_df = pd.DataFrame()
    remark_uas_loc_df['REMARKS'] = remark
    remark_uas_loc_df['UAS_Location_Runways'] = uas_loc
    
    # Get list of UAS locations from the above dataframe
    uas_loc = remark_uas_loc_df['UAS_Location_Runways'] 

    # If the regular expressions did not hit on any location information, pass it UNKN for now
    for i in range(len(remark_uas_loc_df)):
        if len(uas_loc[i]) == 0:
            uas_loc[i] = 'UNKN'
    
    # NOTE :: remark_uas_loc_df is the dataframe that contains ALL records
    #         rwy_df is the dataframe that contains ONLY RECORDS THAT REFERENCES RWY
    
    rwy_df = remark_uas_loc_df[remark_uas_loc_df['UAS_Location_Runways'] != 'UNKN']
    rwy_df = rwy_df.reset_index()
    
    # Read in Output_All_Points.csv from Lex
    output_all_points = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Output_All_Points.csv',encoding='cp1252')
    
    rwy_complete_df = pd.merge(rwy_df, output_all_points, on='REMARKS', how='left')
    rwy_complete_df = rwy_complete_df.drop_duplicates(subset = ['REMARKS'])
    std_rwyComplete = rwy_complete_df[['REMARKS','UAS_Location_Runways', 'UASLOCATION','REPORTINGFACILITY','RWYLOCATION']].reset_index()
    
    # Rename the 'UASLOCATION' field to 'IDENT' for merging purposes
    std_rwyComplete.rename(columns = {'REPORTINGFACILITY': 'IDENT'}, inplace = True)

    # Create List that contains each Standard Remark, and the Heading/Direction
    # information contained in each remark

    remark_uas_loc = []
    remarks = std_rwyComplete['REMARKS']

    # regular expression for any heading/direction
    headir_regex = '\.?[0-9]\.?[0-9]*[0-9]*\s?NM'

    # Loop through all remarks and search for the heading/direction regex above
    for i in range(len(remarks)):
        head_dir = re.findall(headir_regex, remarks[i])
        remark_uas_loc.append(remarks[i])
        remark_uas_loc.append(head_dir)

    # Split Remarks and Heading/Directions into two seperate lists and create
    # pandas dataframe
    remark = []
    uas_loc = []

    for i in range(0, len(remark_uas_loc), 2):
        remark.append(remark_uas_loc[i])
        uas_loc.append(remark_uas_loc[i+1])

    # Append UAS Distance
    std_rwyComplete['UAS_Distance'] = uas_loc
    
    # Export final DF to .csv 
    std_rwyComplete.to_csv('runways%d.csv'%len(std_rwyComplete), index = False)
    
    return std_rwyComplete

In [4]:
df = uas_rwy('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/std_uasLoc_noAir_noNav.csv')
df

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,0,Phoenix PD advised a concerned citizen notifie...,"[RWY8, RWY8]",NaN,NaN,NaN,[4NM]
1,1,Aircraft observed a black UAS off the while E ...,[RWY 08],NaN,NaN,NaN,[5NM]
2,2,Aircraft observed a blue and white UAS over th...,[RWY 19R],.5 NM S of RVS,RVS,RWY 19R,[.5 NM]
3,3,Aircraft observed a UAS 1 NM SW of RWY 26L app...,[RWY 26L],1 NM SW of LAS,LAS,RWY 26L,[1 NM]
4,4,Aircraft observed a UAS off the left side whil...,[RWY 25L],1 NM W of DVT,DVT,RWY 25L,[1 NM]
5,5,0340 INITIAL/CLOSEOUT: Law Enforcement Aircraf...,[RWY8],NaN,NaN,NaN,[1.5 NM]
6,6,Aircraft observed a UAS off the left side whil...,[RWY 26L],1/2 NM W of POC,POC,RWY 26L,[2 NM]
7,7,Aircraft observed a UAS overhead while N bound...,[RWY 35C],1/2 NM S of DFW,DFW,RWY 35C,[2 NM]
8,8,Aircraft observed a UAS off the right side whi...,[RWY 9],NaN,NaN,NaN,[1 NM]
9,9,"Aircraft reported a UAS with red lights at 2,3...",[RWY 35C],6 NM S of END,END,RWY 35C,[6 NM]


In [5]:
def drop_nan(dataframe):
    '''
    This function replaces all instances of [] in the dataframe provided as
    input with NaN and then returns a new dataframe with no 'NaN' values
    
    input : dataframe is a dataframe containing UAS remark information
    output : a new dataframe with no 'NaN' values
    '''
    
    df1 = dataframe.mask(dataframe.applymap(str).eq('[]'))
    output_df = df1.dropna().reset_index(drop=True)
    
    return output_df

In [6]:
df1 = drop_nan(df)
df1

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,2,Aircraft observed a blue and white UAS over th...,[RWY 19R],.5 NM S of RVS,RVS,RWY 19R,[.5 NM]
1,3,Aircraft observed a UAS 1 NM SW of RWY 26L app...,[RWY 26L],1 NM SW of LAS,LAS,RWY 26L,[1 NM]
2,4,Aircraft observed a UAS off the left side whil...,[RWY 25L],1 NM W of DVT,DVT,RWY 25L,[1 NM]
3,6,Aircraft observed a UAS off the left side whil...,[RWY 26L],1/2 NM W of POC,POC,RWY 26L,[2 NM]
4,7,Aircraft observed a UAS overhead while N bound...,[RWY 35C],1/2 NM S of DFW,DFW,RWY 35C,[2 NM]
5,9,"Aircraft reported a UAS with red lights at 2,3...",[RWY 35C],6 NM S of END,END,RWY 35C,[6 NM]


In [7]:
def airRwy_link (dataframe):
    '''
    This function does the following:
        1. Merge the dataframe with the airports_runways_linked.csv dataset
            a. Merge in the IDENT field
        2. Extracts the Runway Designator from RWYLOCATION field
        3. Checks if the Runway Designator found in the Remark matches one
           of the RWY Designators from airports_runways_linked.csv
            a. Only keeps those records for which there is a match
        4. Merge the resulting dataframe with airports_cleaned_declination.csv
           to get the declination for the runway
    
    input : dataframe is a dataframe containing UAS remark information and 
            Runway Location information in a field named RWYLOCATION
    output : a new dataframe in which all records have a runway location
             that is present at an airport in airports_runways_linked.csv
    '''
    # Import and store datasets to merge with
    air_rwy = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_runways_linked.csv')
    air_dec = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned_declination.csv')
    
    # Merge the two datasets on the 'IDENT' field
    rwy_linked = pd.merge(dataframe, air_rwy, on = 'IDENT', how = 'left')
    rwy_linked.rename(columns = {'properties.DESIGNATOR': 'DESIGNATOR'}, inplace = True)
    
    # Extract Runway Designator from RWYLOCATION field
    rwydesignator = []

    for i in range(len(rwy_linked)):
        rwy_split = rwy_linked['RWYLOCATION'][i].split(' ')
        rwydesignator.append(rwy_split[1])

    # Add split column with RWY designators to dataframe as new field
    rwy_linked['RWYDESIGNATOR'] = rwydesignator
    
    # Check if Runway Designator found in Remark matches one of the Runway designators
    # from the air_rwy dataframe
    rwy_linked['RWYinREMARK'] = rwy_linked.apply(lambda x: str(x.RWYDESIGNATOR) in str(x.DESIGNATOR), axis=1)

    # Only keep records for which the RWY Designator is present in the DESIGNATOR field 
    output_df1 = rwy_linked[rwy_linked['RWYinREMARK'] == True]
    
    # Merge resulting dataframe with airport declination (air_dec) dataframe
    output_dec = pd.merge(output_df1, air_dec, on = 'IDENT', how = 'left')
    
    # Only keep necessary columns/fields of interest
    output_df = pd.DataFrame(output_dec[['REMARKS', 'UASLOCATION', 'IDENT', 'RWYLOCATION', 'UAS_Distance', 'RWYDESIGNATOR', 'Runway.Latitude', 'Runway.Longitude', 'DECLINATION']])
    
    return output_df

In [8]:
df2 = airRwy_link(df1)
df2

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway.Latitude,Runway.Longitude,DECLINATION
0,Aircraft observed a blue and white UAS over th...,.5 NM S of RVS,RVS,RWY 19R,[.5 NM],19R,-95.986255,36.038465,2.40077
1,Aircraft observed a UAS 1 NM SW of RWY 26L app...,1 NM SW of LAS,LAS,RWY 26L,[1 NM],26L,-115.147202,36.073604,11.33878
2,Aircraft observed a UAS off the left side whil...,1 NM W of DVT,DVT,RWY 25L,[1 NM],25L,-112.085527,33.687439,10.02699
3,Aircraft observed a UAS off the left side whil...,1/2 NM W of POC,POC,RWY 26L,[2 NM],26L,-117.784134,34.091401,11.60831
4,Aircraft observed a UAS overhead while N bound...,1/2 NM S of DFW,DFW,RWY 35C,[2 NM],35C,-97.026110,32.900981,3.06900
5,"Aircraft reported a UAS with red lights at 2,3...",6 NM S of END,END,RWY 35C,[6 NM],35C,-97.914727,36.345432,3.63203


In [9]:
def rwyUAS_latLong (dataframe):
    '''
    This function takes in a dataframe containing Runway latitude/longitude
    information as well as runway declination information for each Remark
    having a runway origin reference location.
    
    This function does the following:
        1. Calculates bearing information for each runway reference
        2. Converts distances from NM to kilometers
        3. Calculates UAS Latitude/Longitude information using geopy
    
    input : dataframe containing runway lat/long and declination information
    output : dataframe containing UAS Latitude/Longitude information
    '''
    # Calculate bearing information for each runway reference
    designator = dataframe['RWYDESIGNATOR']
    designator_bearing = []

    for i in range(len(dataframe)):
        designator_bearing.append(int(re.sub("\D","",designator[i]))*10 + dataframe['DECLINATION'][i])

    dataframe['RWY_BEARING'] = designator_bearing
    
    # Convert Distances from NM to kilometers
    dist_kilo = []

    for i in range(len(dataframe)):
        distanceKilo = int(re.sub("\D","",str(dataframe['UAS_Distance'][i])))* 1.852 # converting NM to kilometers
        dist_kilo.append(distanceKilo)

    dataframe['Distance_Kilometers'] = dist_kilo
    
    # BECAUSE ERIC HAD THESE SWITCHED ****************
    dataframe.rename(columns = {'Runway.Latitude': 'Runway_Longitude', 'Runway.Longitude' : 'Runway_Latitude'}, inplace = True)
    
    # Calculate UAS Lat/Long information using geopy
    uas_lat = []
    uas_long = []


    for i in range(len(dataframe)):
        lat_rwy = pd.to_numeric(dataframe['Runway_Latitude'][i])
        long_rwy = pd.to_numeric(dataframe['Runway_Longitude'][i])
        b = pd.to_numeric(dataframe['RWY_BEARING'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_rwy, long_rwy)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2, = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)

    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export dataframe containing UAS lat/long information to .csv file 
    dataframe.to_csv('rwy%d_uasLatLong.csv'%len(dataframe), index=False)
    
    return dataframe
    

In [10]:
df3 = rwyUAS_latLong(df2)
df3

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,Aircraft observed a blue and white UAS over th...,.5 NM S of RVS,RVS,RWY 19R,[.5 NM],19R,-95.986255,36.038465,2.40077,192.40077,9.260,35.956956,-96.008298
1,Aircraft observed a UAS 1 NM SW of RWY 26L app...,1 NM SW of LAS,LAS,RWY 26L,[1 NM],26L,-115.147202,36.073604,11.33878,271.33878,1.852,36.073993,-115.167756
2,Aircraft observed a UAS off the left side whil...,1 NM W of DVT,DVT,RWY 25L,[1 NM],25L,-112.085527,33.687439,10.02699,260.02699,1.852,33.684545,-112.105199
3,Aircraft observed a UAS off the left side whil...,1/2 NM W of POC,POC,RWY 26L,[2 NM],26L,-117.784134,34.091401,11.60831,271.60831,3.704,34.092332,-117.824255
4,Aircraft observed a UAS overhead while N bound...,1/2 NM S of DFW,DFW,RWY 35C,[2 NM],35C,-97.026110,32.900981,3.06900,353.06900,3.704,32.934136,-97.030889
5,"Aircraft reported a UAS with red lights at 2,3...",6 NM S of END,END,RWY 35C,[6 NM],35C,-97.914727,36.345432,3.63203,353.63203,11.112,36.444952,-97.928474


In [11]:
def main (dataframe):
    df = uas_rwy(dataframe)
    df1 = drop_nan(df)
    df2 = airRwy_link(df1)
    output = rwyUAS_latLong(df2)
    
    return output

In [13]:
main('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/std_uasLoc_noAir_noNav.csv')

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,Aircraft observed a blue and white UAS over th...,.5 NM S of RVS,RVS,RWY 19R,[.5 NM],19R,-95.986255,36.038465,2.40077,192.40077,9.260,35.956956,-96.008298
1,Aircraft observed a UAS 1 NM SW of RWY 26L app...,1 NM SW of LAS,LAS,RWY 26L,[1 NM],26L,-115.147202,36.073604,11.33878,271.33878,1.852,36.073993,-115.167756
2,Aircraft observed a UAS off the left side whil...,1 NM W of DVT,DVT,RWY 25L,[1 NM],25L,-112.085527,33.687439,10.02699,260.02699,1.852,33.684545,-112.105199
3,Aircraft observed a UAS off the left side whil...,1/2 NM W of POC,POC,RWY 26L,[2 NM],26L,-117.784134,34.091401,11.60831,271.60831,3.704,34.092332,-117.824255
4,Aircraft observed a UAS overhead while N bound...,1/2 NM S of DFW,DFW,RWY 35C,[2 NM],35C,-97.026110,32.900981,3.06900,353.06900,3.704,32.934136,-97.030889
5,"Aircraft reported a UAS with red lights at 2,3...",6 NM S of END,END,RWY 35C,[6 NM],35C,-97.914727,36.345432,3.63203,353.63203,11.112,36.444952,-97.928474
